# Exercises 3 and 4 - Convolutional Neural Networks 

Consider an image which has a filter applied to it. For example, at each pixel, take all the neighbors surrounding it and weight their values and add them all back up together for form a new value for every pixel. This process can add emphasis to certain features of the image, such as vertical or horizontal lines. So, instead of using raw pixel data from the previous examples, the image is processed into features, and the images are classified based on the features they show after filtering.

The process of pooling data decreases resolutions of the images are decreased to use less information and speed up training, while keeping the most important information for detection and classification.

Notebook file: bit.ly/convolutions-fun

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
# convolutions and pooling in code:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

Convolutions and Pooling steps are stacked on top of the dense network 
1. Convolution layer with 64 filters which are randomly initialized and learned over time, each filter has a size of 3x3 pixels, input is 28x28 with a single byte of color depth 
2. Pooling layer, pooling 2x2 pixel chunks based on the largest pixel value in the chunk (max pooling), keeps 1/4 of the information in the image
3. Another convolution and pooling layer
4. The model

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 128)               2

1. Initial output is 26x26? Not 28x28? A 3x3 filter needs pixels with a neighbor on every side of the pixel to work, so it cannot be utilized on the edges and corners of the picture, so these pixels are not used, hence 26x26 shape as the top, right, bottom, and left edges are cut out. Each filter learns 9 values, plus a bias, with a total of 10 values for 640 learnable parameters across 64 filters 
2. Pooling reduces dimensionality by half in each axis, so the pooling layer is 13x13 (half of 26), no parameters are learned in this layer
3. This next convolutional layer has to reduce the number of pixels in each direction by two once again, so this shape is 11x11
4. Pooling halves this again and rounds this down 
5. This process leaves 64 filters for each now 5x5 pixel image, for a total of 1600 values which are flatten, and classified in a dense network 

## Convolutional NN on fashion dataset

In [25]:
data = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()
training_images = training_images / 255.0
test_images = test_images / 255.0

In [27]:
# original network 
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(
    optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']
)

In [28]:
model.fit(training_images, training_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5003 - accuracy: 0.8227
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3745 - accuracy: 0.8644
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3354 - accuracy: 0.8776
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3107 - accuracy: 0.8869
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2941 - accuracy: 0.8926


In [29]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 2ms/step - loss: 0.3427 - accuracy: 0.8759


[0.3426796793937683, 0.8758999705314636]

In [30]:
# convolutional network 
data = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = data.load_data()

# data for the convolutions needs to be reshaped, expects a single tensor containing everything (one 4D array instead of 60000 3D arrays)
training_images = training_images.reshape(60000,28,28,1)
training_images = training_images / 255.0
test_images = test_images.reshape(10000,28,28,1)
test_images = test_images / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']
)
model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 11, 11, 64)        36928     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten_14 (Flatten)        (None, 1600)              0         
                                                                 
 dense_28 (Dense)            (None, 128)             

In [35]:
model.fit(training_images, training_labels, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 69s 37ms/step - loss: 0.1546 - accuracy: 0.9425
Epoch 2/5
1875/1875 [==============================] - 54s 29ms/step - loss: 0.1335 - accuracy: 0.9497
Epoch 3/5
1875/1875 [==============================] - 55s 30ms/step - loss: 0.1161 - accuracy: 0.9560
Epoch 4/5
1875/1875 [==============================] - 69s 37ms/step - loss: 0.1024 - accuracy: 0.9610
Epoch 5/5
1875/1875 [==============================] - 71s 38ms/step - loss: 0.0890 - accuracy: 0.9659


In [36]:
model.evaluate(test_images, test_labels)

313/313 [==============================] - 4s 12ms/step - loss: 0.3405 - accuracy: 0.9106


[0.34045520424842834, 0.9106000065803528]

### Accuracy now much higher accross training and testing data

More Exercises: bit.ly/tfw-lab4